In [103]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [104]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/assignment/prompts_classification_data.csv')

In [105]:
# Encode labels
label_mapping = {label: idx for idx, label in enumerate(df['category'].unique())}
df['label'] = df['category'].map(label_mapping)

In [106]:
# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(df['prompt'], df['label'], test_size=0.2, random_state=42)

In [111]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [112]:
# Tokenize data
def tokenize_data(texts):
    return tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')

train_encodings = tokenize_data(train_texts)
test_encodings = tokenize_data(test_texts)

In [113]:
# Create dataset class
class PromptDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

In [114]:

train_dataset = PromptDataset(train_encodings, train_labels)
test_dataset = PromptDataset(test_encodings, test_labels)

In [115]:
# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [117]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [118]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [119]:
# Train model
trainer.train()

<ipython-input-113-07cd8e96a4b9>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=30, training_loss=3.7153722127278646, metrics={'train_runtime': 10.2925, 'train_samples_per_second': 33.034, 'train_steps_per_second': 2.915, 'total_flos': 2971263898440.0, 'train_loss': 3.7153722127278646, 'epoch': 10.0})

In [120]:
# Evaluate model
test_preds = trainer.predict(test_dataset)
pred_labels = torch.argmax(torch.tensor(test_preds.predictions), axis=1)

<ipython-input-113-07cd8e96a4b9>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.encodings.items()}


In [121]:
# Convert to lists
test_labels_list = test_labels.tolist()
pred_labels_list = pred_labels.tolist()

In [122]:

# Calculate and print accuracy
accuracy = accuracy_score(test_labels_list, pred_labels_list)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.0000


In [124]:
# Extract the unique classes from the test labels and predictions
unique_labels = sorted(set(test_labels_list + pred_labels_list))
target_names = [label for label, idx in sorted(label_mapping.items(), key=lambda item: item[1]) if idx in unique_labels]


In [125]:
# Print classification report
print("Classification Report:")
print(classification_report(test_labels_list, pred_labels_list, labels=unique_labels, target_names=target_names))

Classification Report:
                                 precision    recall  f1-score   support

               Video Generation       0.00      0.00      0.00       1.0
                   Blog Writing       0.00      0.00      0.00       1.0
           Product Descriptions       0.00      0.00      0.00       2.0
                API Integration       0.00      0.00      0.00       0.0
        Legal Document Drafting       0.00      0.00      0.00       1.0
            Contract Generation       0.00      0.00      0.00       1.0
Resume and Cover Letter Writing       0.00      0.00      0.00       1.0
       Job Description Creation       0.00      0.00      0.00       1.0
                 Summarisations       0.00      0.00      0.00       1.0
          Content Summarization       0.00      0.00      0.00       0.0

                       accuracy                           0.00       9.0
                      macro avg       0.00      0.00      0.00       9.0
                   weighte

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.